In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

### Survey Clean file, creating white/black lists

In [2]:
# Read it in
path = "surveys_clean.csv"
df = pd.read_csv(path, na_filter=False)

# Drop unnecessary columns
df.drop(columns=['ind_id', 'subject_id', 'image_name', 'image_name_2', 'index','outfits','outfits_comment'], inplace=True)

In [3]:
whiteDF = df[df['racial_group'] == 'white']['long']
blackDF = df[df['racial_group'] == 'black']['long']

whiteList = [i for i in whiteDF]
blackList = [i for i in blackDF]

In [4]:
whiteList[:5]

['i have no comments.',
 '',
 '',
 'to maney questions a like in the last here.  the rest alright overseas across',
 "why don't a soldiers have the right to take up any kind of work they want in the army."]

In [5]:
blackDF.head()

0    negro have been told many times they're fighti...
1    i dont like the army. i had rather be on the o...
2    i think that if were going to win this war, th...
3    why is it that the negro dont have the the rig...
4    i highly approve of this questionnaire it give...
Name: long, dtype: object

### DEIM-CUR functions

In [6]:
def cur(A, idx=False, SVD=None, rank=None):
    
    if SVD==None:
        SVD = np.linalg.svd(A, full_matrices=False)
    if rank==None:
        rank = np.linalg.matrix_rank(A)
    
    V,S,Wt = SVD
    
    def DEIM(M): # row/column selection
        p = []
        p.append(np.argmax(np.abs(M[:,0]))) # row index of the largest value in the first column of M
        for i in range(1,rank):
            res = M[:,i] - np.dot(M[:,:i],np.dot(np.linalg.inv(M[:,:i][p]), M[:,i][p]))
            p.append(np.argmax(np.abs(res)))
        return p
    
    p_C = DEIM(np.transpose(Wt))
    C = A[:,p_C]
    
    p_R = DEIM(V)
    R = A[p_R]
    
    U = np.linalg.pinv(C) @ A @ np.linalg.pinv(R)
    
    return (C,U,R, p_C, p_R) if idx else (C,U,R)

In [7]:
embed_white = (model.encode(whiteList))
embed_black = (model.encode(blackList))

### Top Responses for black respondents 

In [8]:
C_black,U_black,R_black, p_C_black, p_R_black = cur(np.transpose(embed_black), idx=True)

In [9]:
print("Black\n")
for i in range(20):
    print(blackList[p_C_black[i]])

Black

if the negro soldier is good enough to participate in the tough & rough army to help our country why not have privileges as same as the white man. since we are considered rotten & no good, we shouldn't't have to fight. we realize this is a white man's country but we are willing to fight for it because we live in it. therefore after this war i hope every negro will take this matter into consideration and force less segregation & a fair living as white people have. if all negro were like me things would be a hell of a lot better  site worse. we must show the white man we are all men together before we shall have equal rights. a man is a man no matter what race, creed or color, so why not be treated alike.
i think this questionnaire is good for us.
i don't like army in any way
i want to make more money to take care of my wife and get out the army so i can take care of her and my to kids
we don't get much to eat and we can't get attention our doctors when we are sick, the no  office

### Top Responses for white respondents 

In [10]:
 C_white,U_white,R_white, p_C_white, p_R_white = cur(np.transpose(embed_white), idx=True)

In [11]:
print("\nWhite\n")
for i in range(20):
    print(whiteList[p_C_white[i]])


White

if the army would treat its men as real men and not as boy scouts it would make a man feel like a soldier. they waste too much time on silly and unnecessary inspections. that valuable time could be used for good training. we came here to fight and not to see whether our left shoe is where the right one should be. good inspections are all right but not the kind we have. there have been times in this camp when the morale of the men was so low it was pitiful.

i think the idea of this questionnaire is very good.
no comments.
the army is a good place for every man if he is able to be in the army.
to me i believe that negro should be treated just like us. i would just as soon be in with some negro as some of the american soldiers i have seen.
the soldiers should have better food.
time is limited so i'll have to go back to my job
some of these questions were kind of difficulty to answer in my idea
will this win the war?
i am well sadyes fide in the army
about the two lines in front p

### Spectral Clustering on White respondents <br>
-Using 4 clusters based on spectral gap

In [30]:
embed = np.transpose(embed_white)
first = pd.DataFrame({'Response': whiteList, 'Embedding': embed_white})
second = pd.DataFrame({'Response': whiteList})

In [31]:
first.shape

(2324, 2)

In [32]:
second.shape

(2324, 1)

In [33]:
len(whiteList)

2324

In [34]:
for i in range(0, 768):
    x = []
    for j in range(0, len(embed_white)):
        x.append(first['Embedding'][j][i])
    second['x'+str(i)] = x
    
second.iloc[:,1:]

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x758,x759,x760,x761,x762,x763,x764,x765,x766,x767
0,0.758591,0.456631,1.158288,-0.455719,-0.180095,1.283310,0.706048,0.358347,-0.999309,0.344630,...,-0.227195,0.462643,0.086750,-0.168338,-0.294847,-0.733800,-0.064139,1.485314,-0.084078,0.013112
1,0.309527,-0.083690,1.793920,-0.128317,0.171174,0.389611,-0.056813,0.341523,0.116591,-0.064351,...,0.549659,-0.103774,0.278531,0.671658,-0.126949,-0.480964,0.455517,0.225562,-0.091203,-0.032031
2,0.309527,-0.083690,1.793920,-0.128317,0.171174,0.389611,-0.056813,0.341523,0.116591,-0.064351,...,0.549659,-0.103774,0.278531,0.671658,-0.126949,-0.480964,0.455517,0.225562,-0.091203,-0.032031
3,0.454678,-0.536229,0.779319,0.193302,-0.092970,-0.065367,-0.889141,-0.867361,-0.364727,-0.262388,...,-0.091493,0.120037,0.284678,0.080229,-0.299972,-0.035708,-0.416625,0.467714,-0.377875,0.374595
4,0.422178,0.610620,-0.317593,0.041234,0.122884,0.505356,1.233452,0.438452,-0.306385,-0.208844,...,-0.180563,0.463026,1.149110,-1.301010,-1.116768,0.115643,-0.163776,0.072889,-0.872556,0.422105
5,0.309527,-0.083690,1.793920,-0.128317,0.171174,0.389611,-0.056813,0.341523,0.116591,-0.064351,...,0.549659,-0.103774,0.278531,0.671658,-0.126949,-0.480964,0.455517,0.225562,-0.091203,-0.032031
6,-0.170448,-0.680924,1.901574,0.523110,-0.252908,-0.436074,0.231396,-0.113363,-0.252537,-0.799451,...,-0.771372,-0.200827,0.274970,-1.376745,-0.996408,-0.065467,-0.677858,-0.443991,-0.864252,-0.829374
7,0.549441,-0.032283,0.780587,0.240238,-0.428685,-0.063074,0.501355,-0.903985,0.004191,0.049287,...,-0.264759,0.058556,-0.114252,-1.299849,-0.530832,-0.343054,-0.306447,0.633780,-0.793164,0.330236
8,0.004660,-0.301120,0.862109,0.273910,-0.373713,-0.793335,0.772640,0.344798,1.541640,-0.659954,...,0.481832,-0.242059,0.569427,-0.937235,-0.681376,0.322259,-0.742309,-1.169469,0.090096,-0.443329
9,0.799952,0.370300,0.983754,-0.038101,0.426844,-1.270377,0.507401,-1.155068,0.053498,-0.916051,...,-0.162249,-0.737028,0.351942,-0.329650,-0.667494,0.348835,0.448731,0.669346,-0.273538,0.238912


In [35]:
n=4

from sklearn.cluster import SpectralClustering
spectral = SpectralClustering(n_clusters=n,affinity='nearest_neighbors').fit(second.iloc[:, 1:])
first['Cluster'] = [i for i in spectral.labels_]
first['bag'] = first.Response.map(lambda t:t.replace(';','').lower().split())

In [36]:
first.head()

,Response,Embedding,Cluster,bag
0,i have no comments.,"[0.758591, 0.45663133, 1.1582876, -0.45571896,...",2,"[i, have, no, comments.]"
1,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1,[]
2,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1,[]
3,to maney questions a like in the last here. t...,"[0.4546784, -0.5362293, 0.7793191, 0.19330154,...",0,"[to, maney, questions, a, like, in, the, last,..."
4,why don't a soldiers have the right to take up...,"[0.42217788, 0.6106196, -0.317593, 0.04123385,...",0,"[why, don't, a, soldiers, have, the, right, to..."


In [37]:
whiteClusters = first
whiteClusters.drop(columns=['bag'], inplace=True)
whiteClusters.head()

,Response,Embedding,Cluster
0,i have no comments.,"[0.758591, 0.45663133, 1.1582876, -0.45571896,...",2
1,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
2,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
3,to maney questions a like in the last here. t...,"[0.4546784, -0.5362293, 0.7793191, 0.19330154,...",0
4,why don't a soldiers have the right to take up...,"[0.42217788, 0.6106196, -0.317593, 0.04123385,...",0


### Cluster 0 Top Responses

In [38]:
is_0 = first['Cluster'] == 0
cluster_0 = first[is_0]

cluster_0

,Response,Embedding,Cluster
3,to maney questions a like in the last here. t...,"[0.4546784, -0.5362293, 0.7793191, 0.19330154,...",0
4,why don't a soldiers have the right to take up...,"[0.42217788, 0.6106196, -0.317593, 0.04123385,...",0
6,good thing if higher authorities get some acti...,"[-0.17044759, -0.6809244, 1.9015743, 0.5231100...",0
7,some question overlap others and you might thi...,"[0.549441, -0.032282595, 0.780587, 0.24023785,...",0
8,when is the big drive is goning to start. is ...,"[0.004659554, -0.30111957, 0.8621086, 0.273910...",0
9,i think some types of services in the various ...,"[0.79995203, 0.37030008, 0.9837538, -0.0381007...",0
10,in reference to question #25 there is no coope...,"[-0.4673673, 1.0903834, 0.24132901, 0.14427656...",0
11,let's get something done about prosecuting the...,"[0.18304382, 0.37828875, -0.5551928, 0.1956413...",0
12,i am not fighting to give freedom to the world...,"[0.02518781, 0.24524716, 0.4619027, 0.15978968...",0
13,i want to say that i consider all my efforts i...,"[0.33428708, 0.013279631, 0.61966, 0.085011646...",0


In [51]:
cluster0List = cluster_0['Response'].tolist()

In [52]:
embed0 = np.transpose(model.encode(cluster0List))

In [53]:
C_0,U_0,R_0, p_C_0, p_R_0 = cur(embed0, idx=True)

In [66]:
print("Cluster 0\n")
for i in range(20):
    print(i,':', whiteList[p_C_0[i]],'\n')

Cluster 0

0 : i think they are holding some soldiers in the same camp to long. training most soldiers shouldn't't take more than one year. some in this camp over two years. 

1 : i have none to make. 

2 : 1. why let civilians take up space in our theater, to buy from the px?  2. why not let fellows working on the line have furloughs? they would be much more satisfied if they could see their brothers sisters folks + sweethearts.  3. i've been in the army 10 months, graduated from technical school as power plant specialist on a plane. that work calls for a rating of some kind + fellows that have been in 3 months are corporals taking care of the latrine. we studied hard for seven months to get a specialists rating. why can't we have one? 

3 : i think this  or near by towns could arrange for some entertainment for the soldiers 

4 : i believe this questionnaire is ok & i really don't know what to write or say. 

5 :  

6 : in my humble opinion the army has been handled as well as possib

### Cluster 1 Top Responses

In [59]:
is_1 = first['Cluster'] == 1
cluster_1 = first[is_1]
cluster1List = cluster_1['Response'].tolist()

In [61]:
embed1 = np.transpose(model.encode(cluster1List))

In [62]:
C_1,U_1,R_1, p_C_1, p_R_1 = cur(embed1, idx=True)

In [87]:
print("Cluster 1\n")
for i in range(20):
    if(i<len(p_C_1)):
        print(i,':', whiteList[p_C_1[i]],'\n')

Cluster 1

0 : i have no comments. 

1 : regardless of rank of any man in the army the majority of the men are working to-gather. if a private is doing his best at what he has been assigned (and often he has been given something he knew nothing of before) he is doing as much his part as any non-comm or officer -- the best he can. we are working to-gather and not for each other. if some in authority would realize this they would get better co-operation from the men below them. 



In [80]:
cluster_1

,Response,Embedding,Cluster
1,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
2,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
5,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
16,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
22,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
25,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
27,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
33,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
50,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1
51,,"[0.3095274, -0.08369034, 1.7939203, -0.1283173...",1


### Cluster 2 Top Responses

In [81]:
is_2 = first['Cluster'] == 2
cluster_2 = first[is_2]
cluster2List = cluster_2['Response'].tolist()

In [82]:
embed2 = np.transpose(model.encode(cluster2List))

In [83]:
C_2,U_2,R_2, p_C_2, p_R_2 = cur(embed2, idx=True)

In [86]:
print("Cluster 2\n")
for i in range(20):
    if(i<len(p_C_2)):
        print(i,':', whiteList[p_C_2[i]],'\n')

Cluster 2

0 :  

1 : why don't a soldiers have the right to take up any kind of work they want in the army. 

2 : good thing if higher authorities get some action on this problems. 

3 :  

4 : it is a very good questionnaire if everyone would answer it truthful. one thing wrong it does not cover enough. it should be about 150 questions. as we all know the thought of two races will be for their own. if all questions were answered fairly that would be an excellant examation. 

5 : this whole army works the same as a political machine. that is, those who have the influence have the best jobs, while those without are given all the dirty work to do. most of the officers never seem to forget something which an enlisted man has done wrong and they will always punish him a little harsher because they don't have an open mind anymore towards him. most of all in this camp the pass situation is terrible and i can only get a week-end pass every 7 weeks and i have to struggle to get it. 

6 : i am

### Cluster 3 Responses

In [89]:
is_3 = first['Cluster'] == 3
cluster_3 = first[is_3]
cluster3List = cluster_3['Response'].tolist()

In [90]:
embed3 = np.transpose(model.encode(cluster3List))

In [92]:
C_3,U_3,R_3, p_C_3, p_R_3 = cur(embed3, idx=True)

In [93]:
print("Cluster 3\n")
for i in range(20):
    if(i<len(p_C_3)):
        print(i,':', whiteList[p_C_3[i]],'\n')

Cluster 3

0 : food is good one day and bad next train connections to camp are very bad - while bus service is fairly good - 

1 : i would like to give my opinion on the 45th div. i can't see this outfit going into action. when we go out on a problem every thing seems to be going wrong sometimes i just watch some of the men. i really am afraid when we go into battle the men will be shooting their own men. no cooperation. the trouble with this outfit is there is two cliques the rebels or oklahomans and the yankees. seems as the oklahomans started the div. and are at the head of the whole thing a yank does not have a chance in this outfit to prove his ability any promotions to be given a rebel gets it not for his ability but because he is from oklahoma. and some of the non coms are really dumb. there are plenty of yankee private that should be non coms. 

2 : they say that they place you where you are best suited but that is where they are wrong. i was a machinist in civilian life and tr

## Spectral Clustering on Black Soldiers

In [94]:
embed = np.transpose(embed_black)
first = pd.DataFrame({'Response': blackList, 'Embedding': embed_black})
second = pd.DataFrame({'Response': blackList})

In [95]:
for i in range(0, 768):
    x = []
    for j in range(0, len(embed_black)):
        x.append(first['Embedding'][j][i])
    second['x'+str(i)] = x
    
second.iloc[:,1:]

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x758,x759,x760,x761,x762,x763,x764,x765,x766,x767
0,-0.431953,-0.633117,-0.260997,0.415460,-0.237683,0.366318,0.100023,-0.451081,0.052305,-0.772222,...,-0.642631,0.836087,-0.149190,-1.482470,-0.134439,-0.003614,-0.635169,0.017464,-0.421150,0.702712
1,-0.214992,0.580386,0.443751,0.505816,-0.749663,-0.185046,0.921740,-0.230475,0.477832,-0.794905,...,-0.089814,0.691420,0.742551,-1.448567,-0.545186,0.429773,-0.311271,0.444563,-0.531232,0.895420
2,-0.221185,-0.399868,-0.199939,0.446165,-0.415950,0.128538,0.631262,-0.336796,-0.060227,-0.329619,...,-1.046082,0.840199,-0.032127,-2.288132,-0.212986,0.444507,-0.337037,-0.118294,-0.242437,0.492241
3,-0.295477,0.572017,-0.411632,0.144846,-0.018772,-0.431764,0.487732,-0.857616,-0.205131,-0.364119,...,-0.130924,0.962862,0.074192,-1.474225,0.079387,0.112434,-0.553907,0.097416,-0.755756,0.560328
4,-0.574096,0.517166,0.017153,0.162733,-0.337314,-0.444252,0.936756,-0.192153,0.711736,-0.574696,...,-0.783802,0.350003,-0.138772,-2.146734,-0.595041,-0.228926,0.669961,0.770853,0.078583,0.114617
5,-0.129600,-0.261325,0.835405,0.506072,0.061220,-0.294037,0.120656,-0.810101,0.435580,-1.011258,...,-0.442945,0.436407,-0.291818,-0.086346,0.090718,-0.196374,-0.844386,-0.142229,-0.793373,0.124300
6,0.705506,0.260656,1.377129,-0.589358,-0.328145,-0.258725,-0.291662,0.063210,-0.334792,-0.747061,...,-0.579069,-0.448920,0.896083,0.289612,-0.295455,0.477540,-0.395083,1.179413,0.365323,0.002767
7,-0.100324,0.106731,-0.172789,-0.011465,0.139848,-0.310977,0.551033,-0.638847,0.252459,-0.805329,...,-0.678320,-0.223613,-1.558329,-1.466871,0.271727,-0.165487,-0.295605,0.819695,-0.286685,0.278997
8,-0.162554,0.039374,0.028283,0.265046,0.288264,0.369567,0.081332,-0.402532,-0.118626,-0.996127,...,-0.384398,0.380285,0.070555,-0.870803,0.246333,-0.137658,-0.602829,0.167117,-0.568393,0.324509
9,0.535083,0.356359,1.297027,-0.067209,0.407620,-0.337240,0.114972,-0.310488,0.436291,0.049877,...,-0.105883,-0.432709,0.637354,-0.878139,-0.101677,0.152479,-0.498487,0.130223,-0.645121,0.334355


In [96]:
spectral = SpectralClustering(n_clusters=n,affinity='nearest_neighbors').fit(second.iloc[:, 1:])
first['Cluster'] = [i for i in spectral.labels_]
first['bag'] = first.Response.map(lambda t:t.replace(';','').lower().split())

In [97]:
first.head()

,Response,Embedding,Cluster,bag
0,negro have been told many times they're fighti...,"[-0.43195295, -0.6331174, -0.26099682, 0.41546...",0,"[negro, have, been, told, many, times, they're..."
1,i dont like the army. i had rather be on the o...,"[-0.21499196, 0.58038604, 0.44375092, 0.505815...",0,"[i, dont, like, the, army., i, had, rather, be..."
2,"i think that if were going to win this war, th...","[-0.22118457, -0.39986774, -0.19993901, 0.4461...",0,"[i, think, that, if, were, going, to, win, thi..."
3,why is it that the negro dont have the the rig...,"[-0.29547727, 0.57201684, -0.41163206, 0.14484...",0,"[why, is, it, that, the, negro, dont, have, th..."
4,i highly approve of this questionnaire it give...,"[-0.57409596, 0.5171657, 0.01715277, 0.1627325...",0,"[i, highly, approve, of, this, questionnaire, ..."


In [98]:
blackClusters = first
blackClusters.drop(columns=['bag'], inplace=True)
blackClusters.head()

,Response,Embedding,Cluster
0,negro have been told many times they're fighti...,"[-0.43195295, -0.6331174, -0.26099682, 0.41546...",0
1,i dont like the army. i had rather be on the o...,"[-0.21499196, 0.58038604, 0.44375092, 0.505815...",0
2,"i think that if were going to win this war, th...","[-0.22118457, -0.39986774, -0.19993901, 0.4461...",0
3,why is it that the negro dont have the the rig...,"[-0.29547727, 0.57201684, -0.41163206, 0.14484...",0
4,i highly approve of this questionnaire it give...,"[-0.57409596, 0.5171657, 0.01715277, 0.1627325...",0


### Cluster 0 Top Responses (Black)

In [99]:
is_0 = first['Cluster'] == 0
cluster_0 = first[is_0]
cluster0List = cluster_0['Response'].tolist()

In [100]:
cluster_0.shape

(3239, 3)

In [101]:
embed0 = np.transpose(model.encode(cluster0List))

In [102]:
C_0,U_0,R_0, p_C_0, p_R_0 = cur(embed0, idx=True)

In [104]:
print("Cluster 0\n")
for i in range(20):
    if(i<len(p_C_0)):
        print(i,':', blackList[p_C_0[i]],'\n')

Cluster 0

0 : the questionnaire itself was a good one. will it be like other things that are good for the negro, be forgotten after a discussion? 

1 : i think this questionnaire should  unusual attention, because those kinds of questions enable's whoever reads them to find out just how the negro are being treated. in my opinion i think if we are to help fight this war why not not treat us as a soldier with a hell of a job to do. i think camp claiborn louisiana as one of the lously place's in the world to train a negro soldier. in my outfit about 90% of the boys are from the north, and are use to being treated as a citizon of this amercian. frankly speaky we have come to the conclusion that we are just now fighting for what (?) (in a white man's army.) so why not give us a break, we can soldier as good or better than any white man ever dare to. were doing the same as he, why not treat us as they are being treated colored soldiers are not treated fair in the south. colored soldiers are

### Cluster 1 Top Responses (Black)

In [105]:
is_1 = first['Cluster'] == 1
cluster_1 = first[is_1]
cluster1List = cluster_1['Response'].tolist()

In [106]:
cluster_1.shape

(19, 3)

In [107]:
embed1 = np.transpose(model.encode(cluster1List))

In [108]:
C_1,U_1,R_1, p_C_1, p_R_1 = cur(embed1, idx=True)

In [109]:
print("Cluster 1\n")
for i in range(20):
    if(i<len(p_C_1)):
        print(i,':', blackList[p_C_1[i]],'\n')

Cluster 1

0 : negro have been told many times they're fighting this war so that they may have democracy and the same rights and privileges as whites. but a colored soldier is more discriminated against and segregated than he was when he was a civilian. its evident to those who care to see it that not many of the promises to negro are being kept. since the emancipation proclamation negro have been fighting for what they have now. each right, each privilege was fought for. whatever conclusions may be drawn from this survey lets hope its for the further advantage of the negro. negro love america and will fight for america but the negro are going to demand a status as a citizen equal to that of a white citizen. only that way can an example of democracy be used for other countries. 

1 : think we going to win this war 

2 : why is it that the negro dont have the the rights as the white soldier in traveling around in towns as the white when they all are just the same so far as brotherhood i

### Cluster 2 Top Responses (Black)

In [110]:
is_2 = first['Cluster'] == 2
cluster_2 = first[is_2]
cluster2List = cluster_2['Response'].tolist()

In [111]:
cluster_2.shape

(26, 3)

In [112]:
embed2 = np.transpose(model.encode(cluster2List))

In [113]:
C_2,U_2,R_2, p_C_2, p_R_2 = cur(embed2, idx=True)

In [114]:
print("Cluster 2\n")
for i in range(20):
    if(i<len(p_C_2)):
        print(i,':', blackList[p_C_2[i]],'\n')

Cluster 2

0 : above all lets stick together and beat the axis as soon as possible so as to get back to normal living again. other problems concerning negro or whites rights should be thrashed out in congress with represen-tation of both races present. 

1 : the infantry is all right but it is to much walking. 

2 : i am a negro soldier i would like very much to get ahead but my phiscal condition will not premit it. but with a little medical attention i could be outfit and able to get ahead but in my present condition i am not outfit for any position in any branch of the armed forces. 

3 : to my idea i do thank as a united  we should come to gather. i am under the same rule and regulation the white man is. and i am human the same as he i have to live just like dose. and we do as a colored race obey the law better than he dose. how would you like for the colored race. two take your mother out and hang her to a tree and beet her to death. are your wife sister brother are farther 

4 : i

###  Cluster 3 Top Responses (Black)

In [115]:
is_3 = first['Cluster'] == 3
cluster_3 = first[is_3]
cluster3List = cluster_3['Response'].tolist()

In [116]:
cluster_3.shape

(180, 3)

In [117]:
embed3 = np.transpose(model.encode(cluster3List))

In [118]:
C_3,U_3,R_3, p_C_3, p_R_3 = cur(embed3, idx=True)

In [119]:
print("Cluster 3\n")
for i in range(20):
    if(i<len(p_C_3)):
        print(i,':', blackList[p_C_3[i]],'\n')

Cluster 3

0 : the army's ok and you can send me to  in the morning if you wish to. 

1 : to speak very frank both races are fighting for the same purpose. i can't see any reason why there should be any prejudice at all. all are human being's. a colored mans life means just as much to him as a white's. a negro soldier thinks just as much of his families as a white. that's one reason why this country has so many difficulties, they have a war within the united states. i can't understand why some of our distinguished leaders don't wake up and realize that "all men are created equal," and should be treated likewise. 

2 : above all lets stick together and beat the axis as soon as possible so as to get back to normal living again. other problems concerning negro or whites rights should be thrashed out in congress with represen-tation of both races present. 

3 : why should we get no trip home then we do begise  of us have not gen home sent we been here some of us be here for a year more an 